In [ ]:
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import optimizers
from tensorflow.python.keras import activations
from tensorflow.python.keras import callbacks
from tensorflow.python.keras import initializers
from tensorflow.python.keras import backend
from tensorflow.python.keras import metrics
from tensorflow.python import keras
import tensorflow as tf
import keras_tuner as kt
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
train_x_rc = np.load('../data/train_x_rc.npy')
train_x_other = np.load('../data/train_x_other.npy')
targets = np.load('../data/target.npy')
print(train_x_other.shape)
print(train_x_rc.shape)
print(targets.shape)

In [ ]:
class ExpandTileLayer(layers.Layer):
    def __init__(self):
        super(ExpandTileLayer, self).__init__()
    def call(self, inputs, *args, **kwargs):
        return backend.tile(backend.expand_dims(inputs, axis=-2), (1, 80, 1))

In [ ]:
def build_model(hp):
    rc_input = keras.Input(shape=(train_x_rc.shape[-1], ),
                           dtype='int32',
                           name='rc_input_layer')
    other_x_input = keras.Input(shape=(80, train_x_other.shape[-1]),
                                dtype='float32',
                                name='other_x_input')
    rc_embedding_layer = layers.Dense(units=hp.Choice('units_emb', [8, 16, 32, 64]),
                                      use_bias=False, name='rc_embed_layer')
    rc_embedding = rc_embedding_layer(rc_input)

    # 维度拓展
    rc_embedding = ExpandTileLayer()(rc_embedding)

    x_input = layers.Concatenate(axis=-1)([other_x_input, rc_embedding])
    ox = layers.Bidirectional(layers.LSTM(units=hp.Choice('units_0', [32, 64, 128, 256]),
                                          return_sequences=True,
                                          kernel_initializer=initializers.initializers_v2.GlorotUniform()),
                              merge_mode='concat')(x_input)
    for i in range(hp.Int('num_layers', min_value=1, max_value=3, step=1)):
        ox = layers.Bidirectional(layers.LSTM(units=hp.Choice('units_'+str(i), [32, 64, 128, 256]),
                                              return_sequences=True,
                                              kernel_initializer=initializers.initializers_v2.GlorotUniform()),
                                  merge_mode='concat')(ox)
    ox = layers.Dense(units=hp.Choice('units_d1', [32, 64, 128]),
                      kernel_initializer=initializers.initializers_v2.GlorotUniform())
    if hp.Choice('d2_activation', ['prelu', 'elu']) == 'prelu':
        lstm_output = layers.PReLU()(ox)
    else:
        lstm_output = layers.ELU()(ox)

    conv1d_1 = layers.Conv1D(filters=hp.Choice('conv1d_1_filter', [64, 128, 256]),
                             kernel_size=hp.Choice('cov1d_1_size', [5, 11]),
                             padding='same', use_bias=False)(x_input)
    conv1d_1 = layers.BatchNormalization()(conv1d_1)
    conv1d_1 = layers.ReLU()(conv1d_1)
    conv1d_2 = layers.Conv1D(filters=64, padding='same', use_bias=False,
                             kernel_size=hp.Choice('conv1d_2_size', [5, 7]))(conv1d_1)
    conv1d_2 = layers.BatchNormalization()(conv1d_2)
    conv1d_output = layers.ReLU()(conv1d_2)

    output = layers.Concatenate(axis=-1)([lstm_output, conv1d_output])
    output = layers.Dense(units=hp.Choice('final_units', [128, 256]),
                          activation=activations.relu,
                          kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)
    output = layers.Dense(units=1,
                          kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)

    my_model = models.Model(inputs=[rc_input, other_x_input], outputs=[output])
    my_model.compile(loss=losses.MeanAbsoluteError(),
                     optimizer=optimizers.adam_v2,
                     metrics=[metrics.MeanAbsoluteError()])

    print(my_model.summary())
    